# Othello Computer Vision Analysis for Real World

In [8]:
import cv2
import numpy as np


"""
Connect 4
X = blue = player 1
O = red = player 2

Othello
X = B = black = player 1
O = W = white = player 2
"""

# Constants for the board dimensions (number of squares)
BOARD_WIDTH = 8
BOARD_HEIGHT = 8

def grid_to_position_string(grid):
    rows, cols = grid.shape  # Assuming grid is a numpy array with .shape attribute
    position_string = ''

    for col in range(cols):
        for row in range(rows):
            if grid[row][col] == 1:
                position_string += 'B'
            elif grid[row][col] == -1:
                position_string += 'W'
            else:
                position_string += '-'

    return position_string

# Main
def process_frame(frame):
    img = frame

    # Constants
    new_width = 500
    img_h, img_w, _ = img.shape
    scale = new_width / img_w
    img_w = int(img_w * scale)
    img_h = int(img_h * scale)
    img = cv2.resize(img, (img_w, img_h), interpolation=cv2.INTER_AREA)
    img_orig = img.copy()

    # Bilateral Filter
    bilateral_filtered_image = cv2.bilateralFilter(img, 15, 190, 190)

    # Calculate the size of each grid cell
    cell_width = img_w // BOARD_WIDTH
    cell_height = img_h // BOARD_HEIGHT

    # Create a copy of the original image to draw the grid
    grid_image = img_orig.copy()

    # Draw vertical lines for the grid
    for i in range(1, BOARD_WIDTH):
        cv2.line(grid_image, (i * cell_width, 0), (i * cell_width, img_h), (0, 255, 0), 1)

    # Draw horizontal lines for the grid
    for i in range(1, BOARD_HEIGHT):
        cv2.line(grid_image, (0, i * cell_height), (img_w, i * cell_height), (0, 255, 0), 1)

    # Display the image with the grid
    cv2.imwrite('grid_image_with_cells.png', grid_image)

    # Initialize the grid
    grid = np.zeros((BOARD_HEIGHT, BOARD_WIDTH))

    # Constants for color detection for black and white pieces RGB bounds!
    BLACK_LOWER_HSV = np.array([0, 0, 0])
    BLACK_UPPER_HSV = np.array([40, 40, 40])      # previously 110, 110, 110
    WHITE_LOWER_HSV = np.array([150, 150, 150])
    WHITE_UPPER_HSV = np.array([255, 255, 255])
    
    # Create masks for white and black pieces
    white_mask = cv2.inRange(bilateral_filtered_image, WHITE_LOWER_HSV, WHITE_UPPER_HSV)
    cv2.imwrite('white_mask.png', white_mask)

    black_mask = cv2.inRange(bilateral_filtered_image, BLACK_LOWER_HSV, BLACK_UPPER_HSV)
    cv2.imwrite('black_mask.png', black_mask)
    
    game_pieces_mask = black_mask + white_mask
    cv2.imwrite('game_pieces_mask.png', game_pieces_mask)

    not_game_pieces_mask = cv2.bitwise_not(game_pieces_mask)

    # Need to mask away the red, green, and yellow colors as well
    RED_MASK_LOWER = np.array([80, 0, 0])
    RED_MASK_UPPER = np.array([150, 50, 50])
    GREEN_MASK_LOWER = np.array([0, 110, 0])
    GREEN_MASK_UPPER = np.array([30, 150, 30])
    YELLOW_MASK_LOWER = np.array([216, 216, 39])
    YELLOW_MASK_UPPER = np.array([255, 255, 77])
    red_mask = cv2.inRange(bilateral_filtered_image, RED_MASK_LOWER, RED_MASK_UPPER)
    cv2.imwrite('red_mask.png', red_mask)
    green_mask = cv2.inRange(bilateral_filtered_image, GREEN_MASK_LOWER, GREEN_MASK_UPPER)
    cv2.imwrite('green_mask.png', green_mask)
    yellow_mask = cv2.inRange(bilateral_filtered_image, YELLOW_MASK_LOWER, YELLOW_MASK_UPPER)
    
    # Background color masking
    BOARD_BACKGROUND_LOWER = np.array([70, 60, 50])
    BOARD_BACKGROUND_UPPER = np.array([90, 255, 255])
    
    # Create a mask for the background color of the board
    background_mask = cv2.inRange(bilateral_filtered_image, BOARD_BACKGROUND_LOWER, BOARD_BACKGROUND_UPPER)

    # Invert the background mask to remove the background color
    background_mask = cv2.bitwise_not(background_mask)

    # Combine masks to create a single mask representing areas to be removed
    unwanted_mask = not_game_pieces_mask + red_mask + green_mask + yellow_mask
    cv2.imwrite('unwanted_mask.png', unwanted_mask)

    # Apply the background mask to remove the background color
    result_image = cv2.bitwise_and(bilateral_filtered_image, bilateral_filtered_image, mask=background_mask)
    result_image = cv2.bitwise_and(bilateral_filtered_image, bilateral_filtered_image, mask=unwanted_mask)

    # Combine the result with the game pieces mask to identify the game pieces
    result_image = cv2.bitwise_and(bilateral_filtered_image, bilateral_filtered_image, mask=unwanted_mask)
    result_image = cv2.bitwise_or(result_image, result_image, mask=not_game_pieces_mask)
    cv2.imwrite('result_image.png', result_image)


    # Function to check if the majority of the pixels in a masked area are of the color of the mask
    def is_color_dominant(mask):
        # Count the non-zero (white) pixels in the mask
        white_pixels = cv2.countNonZero(mask)
        # Calculate the percentage of white pixels
        white_area_ratio = white_pixels / mask.size
        # If the white area covers more than 40% of the mask, we consider the color to be dominant
        return white_area_ratio > 0.4


    def process_cell(img, x_start, y_start, width, height):
        # Crop the cell from the image
        cell_img = img[y_start:y_start + height, x_start:x_start + width]
        cv2.imwrite('cell_img.png', cell_img)

        # Create masks for white and black pieces
        white_mask_cell = cv2.inRange(cell_img, WHITE_LOWER_HSV, WHITE_UPPER_HSV)
        cv2.imwrite('white_mask_cell.png', white_mask_cell)
        black_mask_cell = cv2.inRange(cell_img, BLACK_LOWER_HSV, BLACK_UPPER_HSV)
        cv2.imwrite('black_mask_cell.png', black_mask_cell)

        if is_color_dominant(white_mask_cell):
            print("White piece detected")
            return -1  # White 
        elif is_color_dominant(black_mask_cell):
            print("Black piece detected")
            return 1  # Black 
        else:
            # The space is empty
            return 0

    # CONTINUE PROCESS FRAME
    # Analyze each cell and update the grid
    for row in range(BOARD_HEIGHT):
        for col in range(BOARD_WIDTH):
            x_start = col * cell_width
            y_start = row * cell_height
            grid[row, col] = process_cell(bilateral_filtered_image, x_start, y_start, cell_width, cell_height)
    
    return grid

# determines if there is motion in the frame
backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)

# Could be better...
def is_motion(previous_frame, current_frame, threshold=10):
    frame_delta = cv2.absdiff(previous_frame, current_frame)
    thresholded = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
    motion_level = np.sum(thresholded)
    return motion_level > threshold

def board_is_full(grid):
    for row in grid:
        for cell in row:
            if cell == 0:
                return False
    return True


def check_winner(grid):
    rows, cols = len(grid), len(grid[0])
    grid_array = np.array(grid)

    # If the board is not full then we have not found a winner
    if (not board_is_full(grid_array)):
        return 0
    
    black_count = 0
    white_count = 0
    for row in grid_array:
        for cell in grid_array:
            if cell == 1:
                black_count += 1
            elif cell == -1:
                white_count += 1

    # If black has more tiles then black wins
    if black_count > white_count:
        return 1
    elif white_count > black_count:
        return -1
    else:
        return 0


def extract_frames(video_path, skip_frames=20):
    cap = cv2.VideoCapture(video_path)
    previous_position_string = None
    output_strings = []  # List to store output strings

    ret, previous_frame = cap.read()
    if not ret:
        output_strings.append("Error: Cannot read frame from video.")
        cap.release()
        return output_strings
    
    previous_frame = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)
    previous_frame = cv2.GaussianBlur(previous_frame, (21, 21), 0)
    previous_position_string = '------------------------------------------'
    p, frame_count = 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        if frame_count % skip_frames == 0:
            if not is_motion(previous_frame, gray):
                board_array = process_frame(frame)
                current_position_string = grid_to_position_string(board_array)
                
                # There is a change between one game state and the next
                if current_position_string != previous_position_string:
                    output_strings.append(f'p={str((p % 2) + 1)}_{current_position_string}')
                    p += 1

                # result = check_winner(board_array)
                # if result == -1:
                #     output_strings.append("Player 2 WHITE wins!")
                #     break
                # elif result == 1:
                #     output_strings.append("Player 1 BLACK wins!")
                #     break

                previous_position_string = current_position_string
            previous_frame = gray
        frame_count += 1

    # Check the winner at the end due to the win condition of othello
    

    cap.release()
    return output_strings

print(extract_frames('uploads/input-othello-8x8-real-world-championship.mov'))

White piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
White piece detected
White piece detected
Black piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece detected
White piece d